In [1]:
import pandas as pd

In [2]:
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [5]:

data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\Flight_Ticket_Price_Prediction\notebooks\data\flight.csv')

In [6]:
cat=[]
num=[]
for i in data.columns:
    if data[i].dtype=='object':
        cat.append(i)
    else:
        num.append(i)

In [7]:
data_cat=data[cat]

In [8]:
data_num=data[num]

In [9]:
for i in data_cat.columns:
    x=pd.unique(data_cat[i])
    print(i,"=",x)

airline = ['SpiceJet' 'AirAsia' 'Vistara' 'GO_FIRST' 'Indigo' 'Air_India']
flight = ['SG-8709' 'SG-8157' 'I5-764' ... '6E-7127' '6E-7259' 'AI-433']
source_city = ['Delhi' 'Mumbai' 'Bangalore' 'Kolkata' 'Hyderabad' 'Chennai']
departure_time = ['Evening' 'Early_Morning' 'Morning' 'Afternoon' 'Night' 'Late_Night']
stops = ['zero' 'one' 'two_or_more']
arrival_time = ['Night' 'Morning' 'Early_Morning' 'Afternoon' 'Evening' 'Late_Night']
destination_city = ['Mumbai' 'Bangalore' 'Kolkata' 'Hyderabad' 'Chennai' 'Delhi']
class = ['Economy' 'Business']


In [10]:
for  j in data_cat.columns:
    data_cat[j]=data_cat[j].astype('category').cat.codes

In [11]:
data=pd.concat([data_cat,data_num],axis=1)

In [12]:
y=data.iloc[:,-1:]

In [13]:
x=data.iloc[:,:-1]

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
from sklearn.model_selection import train_test_split,cross_val_score

In [16]:
from sklearn.linear_model import ElasticNet

In [17]:
import optuna

In [18]:
s=StandardScaler()

In [19]:
x_scaled=s.fit_transform(x)

In [20]:
x_scaled=pd.DataFrame(x_scaled,columns=x.columns)

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=.2,random_state=12)

In [22]:
from sklearn.metrics import r2_score

In [23]:
def objective(trial):
    alpha=trial.suggest_float('alpha',.1,10,log=True)
    l1_ratio=trial.suggest_float('l1_ratio',.1,1,log=True)
    max_iter=trial.suggest_int('max_iter',1000,10000)
    model=ElasticNet(alpha=alpha,
    l1_ratio=l1_ratio,
    max_iter=max_iter
    )
    model.fit(x_train,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [24]:
study=optuna.create_study(direction='maximize')

[I 2024-04-05 00:18:11,648] A new study created in memory with name: no-name-e1975e75-c336-4e2f-a296-f79aa4456704


In [25]:
study.optimize(objective,n_trials=100)

[I 2024-04-05 00:18:23,922] Trial 0 finished with value: 0.8524466215776527 and parameters: {'alpha': 0.38941772034419203, 'l1_ratio': 0.10273728697841474, 'max_iter': 3678}. Best is trial 0 with value: 0.8524466215776527.
[I 2024-04-05 00:18:24,622] Trial 1 finished with value: 0.5491177882942742 and parameters: {'alpha': 3.969417723950345, 'l1_ratio': 0.28508966226632176, 'max_iter': 9935}. Best is trial 0 with value: 0.8524466215776527.
[I 2024-04-05 00:18:25,614] Trial 2 finished with value: 0.8446306850371533 and parameters: {'alpha': 0.49990914964988015, 'l1_ratio': 0.2039066839256325, 'max_iter': 4111}. Best is trial 0 with value: 0.8524466215776527.
[I 2024-04-05 00:18:25,742] Trial 3 finished with value: 0.5738901615234979 and parameters: {'alpha': 6.446442912566151, 'l1_ratio': 0.6037053683465234, 'max_iter': 2542}. Best is trial 0 with value: 0.8524466215776527.
[I 2024-04-05 00:18:25,789] Trial 4 finished with value: 0.30562986799357517 and parameters: {'alpha': 9.815074899

In [26]:
study.best_params

{'alpha': 0.1087747026545574, 'l1_ratio': 0.9916834803032861, 'max_iter': 7634}

In [27]:
study.best_value

0.9046459074231786

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
r=RandomForestRegressor()

In [30]:
r.fit(x_train,y_train)

RandomForestRegressor()

In [31]:
p=r.predict(x_test)

In [32]:
r2_score(y_test,p)

0.9913492278330446